# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

## 1. Introduction

Lima is the capital city of Peru, furthermore is considered as a political, cultural, commercial and financial center of the country. Which has diverse culture, comes diverse food items. There are many restaurants in Lima, each beloging to different categories like Chenese, Italian, Egyptian, etc. In the other hand, studies shows that peruavian people like to eat in the restaurants. They love to eat hamburgers at night. So as a part of this project, I will list and visualise all districts of Lima city that has great opportunity to open a new hamburguer restaurant.

## 1.1 Description of the problem.

There are many Districs in Lima city, 50 to be precise. So If you're planning to open a new hamburguer restaurant and you want to know the top 5 districs with potencial customers, you are in the right page.

## 1.2 Description of the data and how it will be used to solve the problem

For this project I used this page for scraping the relevant information: https://es.wikipedia.org/wiki/Lima_Metropolitana and then explore the districs in Lima city.

## 2. Data preparation

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [6]:
#import library to obatain the data
import requests
#get the entire html of the url as a str
wikipedia_url = requests.get('https://es.wikipedia.org/wiki/Lima_Metropolitana').text

#import library for web scraping
from bs4 import BeautifulSoup
lima_data = BeautifulSoup(wikipedia_url, 'html.parser') #transform the text to html

#find a table in wikipedia page
lima_info_table = lima_data.find('table', class_ = 'wikitable')
lima_rows = lima_info_table.find_all('tr')

# extract the info ('Ubicacion', 'Distrito', 'Poblacion(habitantes)') from the table
lima_info = []
for row in lima_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    lima_info.append(info)
    
lima_info[0:10]

[['Ubicación', 'Distrito', 'Población (habitantes )'],
 ['070701', 'Callao', '426\xa0649'],
 ['070702', 'Bellavista', '78\xa0489'],
 ['070703', 'Carmen de La Legua', '43\xa0156'],
 ['070704', 'La Perla', '64\xa0111'],
 ['070705', 'La Punta', '3955'],
 ['070706', 'Mi Perú', '52\xa0722'],
 ['070707', 'Ventanilla', '356\xa0040'],
 ['150101', 'Lima', '276\xa0861'],
 ['150102', 'Ancón', '43\xa0951']]

In [43]:
#transform the table into dataframe
lima_info[0][-1] = 'Poblacion' # change to american spelling
lima_df = pd.DataFrame(lima_info[1:], columns=lima_info[0])

lima_df.head(10)

,Ubicación,Distrito,Poblacion
0,070701,Callao,426 649
1,070702,Bellavista,78 489
2,070703,Carmen de La Legua,43 156
3,070704,La Perla,64 111
4,070705,La Punta,3955
5,070706,Mi Perú,52 722
6,070707,Ventanilla,356 040
7,150101,Lima,276 861
8,150102,Ancón,43 951
9,150103,Ate,638 345


In [45]:
lima_df.shape

(50, 3)

In [46]:
# adding the prefix Lima to all districts. I dit it because i want to be more precise with the coordinates
lima_df['Distrito'] = lima_df['Distrito'].apply(lambda x: "{}{}".format('Lima, ', x))
lima_df.head()

,Ubicación,Distrito,Poblacion
0,070701,"Lima, Callao",426 649
1,070702,"Lima, Bellavista",78 489
2,070703,"Lima, Carmen de La Legua",43 156
3,070704,"Lima, La Perla",64 111
4,070705,"Lima, La Punta",3955


In [47]:
# use of geopy geocoders to get the coordinates of all districts
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="lima_explorer")
lima_df['Major_Dist_Coord']= lima_df['Distrito'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
lima_df[['Latitude', 'Longitude']] = lima_df['Major_Dist_Coord'].apply(pd.Series)
#remove column 'Major_Dist_Coord'
lima_df.drop(['Major_Dist_Coord'], axis=1, inplace=True)
lima_df.head()

,Ubicación,Distrito,Poblacion,Latitude,Longitude
0,070701,"Lima, Callao",426 649,-12.003654,-77.119244
1,070702,"Lima, Bellavista",78 489,-12.059736,-77.111161
2,070703,"Lima, Carmen de La Legua",43 156,-12.039379,-77.090561
3,070704,"Lima, La Perla",64 111,-12.071343,-77.117564
4,070705,"Lima, La Punta",3955,-12.070896,-77.162264


In [10]:
# lets check all the districts in Lima
lima_df['Distrito'].unique()

array(['Lima, Callao', 'Lima, Bellavista', 'Lima, Carmen de La Legua',
       'Lima, La Perla', 'Lima, La Punta', 'Lima, Mi Perú',
       'Lima, Ventanilla', 'Lima, Lima', 'Lima, Ancón', 'Lima, Ate',
       'Lima, Barranco', 'Lima, Breña', 'Lima, Carabayllo',
       'Lima, Cieneguilla', 'Lima, Chaclacayo', 'Lima, Chorrillos',
       'Lima, Comas', 'Lima, El Agustino', 'Lima, Independencia',
       'Lima, Jesús María', 'Lima, La Molina', 'Lima, La Victoria',
       'Lima, Lince', 'Lima, Los Olivos', 'Lima, Lurigancho',
       'Lima, Lurín', 'Lima, Magdalena del Mar', 'Lima, Miraflores',
       'Lima, Pachacámac', 'Lima, Pucusana', 'Lima, Pueblo Libre',
       'Lima, Puente Piedra', 'Lima, Punta Hermosa', 'Lima, Punta Negra',
       'Lima, Rímac', 'Lima, San Bartolo', 'Lima, San Borja',
       'Lima, San Isidro', 'Lima, San Juan de Lurigancho',
       'Lima, San Juan de Miraflores', 'Lima, San Luis',
       'Lima, San Martín de Porres', 'Lima, San Miguel',
       'Lima, Santa Anita', 'Li

In [48]:
# I wanted to eliminate the white space between poblacion numbers
lima_df['Poblacion'] = lima_df['Poblacion'].str.replace(" ","")
lima_df = lima_df.sort_values(['Poblacion'], axis=0, ascending = True)
lima_df.head(10)

,Ubicación,Distrito,Poblacion,Latitude,Longitude
36,150130,"Lima, San Borja",114 688,-12.096452,-76.995690
28,150122,"Lima, Pachacámac",131 037,-12.250682,-76.906646
42,150136,"Lima, San Miguel",138 226,-12.078656,-77.095283
44,150138,"Lima, Santa María del Mar",1638,-12.401403,-76.775465
34,150128,"Lima, Rímac",167 617,-12.020304,-77.035463
21,150115,"Lima, La Victoria",174 958,-12.073358,-77.016417
20,150114,"Lima, La Molina",175 237,-12.090177,-76.922338
29,150123,"Lima, Pucusana",17 340,-12.482092,-76.797453
17,150111,"Lima, El Agustino",194 474,-12.042052,-76.995714
45,150139,"Lima, Santa Rosa",19 047,-12.035851,-77.086616


So as the next step we will use Foursquare data and obtain information on restaurants. With these, we can start with our battle of neighborhoods for opening a restaurant in Lima, Peru.

Let's visualizate Lima.

In [49]:
# create map of Lima using latitude and longitude values
map_sm = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(lima_df['Latitude'], lima_df['Longitude'], lima_df['Distrito']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sm)  
    
map_sm

## 3. Methodology

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version

In [50]:
CLIENT_ID = 'SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04' # your Foursquare ID
CLIENT_SECRET = 'RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04
CLIENT_SECRET:RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT


In [51]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04&client_secret=RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT&v=20180605&ll=-12.00365435,-77.1192437375166&radius=500&limit=100'

In [52]:
results = requests.get(url).json()

In [53]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pista 15 AIJCH,Airport,-12.007407,-77.120126
1,Sacsayhuaman,Breakfast Spot,-12.005463,-77.118210
2,Av. Faucett y Av. Canta Callao,Bus Stop,-12.000741,-77.116580


In [55]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [56]:
# function to repeat the exploring process to all the districts in Lima
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [57]:
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04' # your Foursquare ID
CLIENT_SECRET = 'RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Analyze Each Neighborhood

Now we are able to check all the restaurants, schools, universitys per districts to know which districts have more potential clients.

In [58]:
# Use category id 4bf58dd8d48988d16c941735 to only get the burger joints
lima_venues_burger = getNearbyVenues(names=lima_df['Distrito'], latitudes=lima_df['Latitude'], longitudes=lima_df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
lima_venues_burger.head(20)

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lima, San Borja",-12.096452,-76.995690,Bembos,-12.096483,-77.002567,Burger Joint
1,"Lima, San Borja",-12.096452,-76.995690,Bembos,-12.097452,-76.990219,Burger Joint
2,"Lima, San Borja",-12.096452,-76.995690,McArthur,-12.090125,-76.996221,Burger Joint
3,"Lima, San Borja",-12.096452,-76.995690,Crispy Gourmet,-12.091875,-77.003195,Burger Joint
4,"Lima, San Borja",-12.096452,-76.995690,Sandwich Campeon,-12.102774,-77.001286,Burger Joint
5,"Lima, San Borja",-12.096452,-76.995690,Kims,-12.102424,-76.994224,Burger Joint
6,"Lima, San Borja",-12.096452,-76.995690,El Lomito,-12.098696,-77.001828,Burger Joint
7,"Lima, Pachacámac",-12.250682,-76.906646,D'Paso,-12.255553,-76.913192,Burger Joint
8,"Lima, Pachacámac",-12.250682,-76.906646,Chicharroneria mamacona,-12.255152,-76.913327,Burger Joint
9,"Lima, San Miguel",-12.078656,-77.095283,Milonga's Sandwichs,-12.077772,-77.096356,Burger Joint


In [25]:
lima_venues_burger.shape

(330, 7)

In [59]:
# function to add markers for given venues to map
def addToMap(lima_df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(lima_df['Venue Latitude'], lima_df['Venue Longitude'], lima_df['Localidad'], lima_df['Venue'], lima_df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [27]:
map_lima_burger = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(lima_venues_burger, 'red', map_lima_burger)
map_lima_burger

With this category ID '4bf58dd8d48988d13d941735' we can visualize the schools per districts

In [30]:
lima_venues_highschools = getNearbyVenues(names=lima_df['Distrito'], latitudes=lima_df['Latitude'], longitudes=lima_df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
lima_venues_highschools.head(20)

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lima, San Borja",-12.096452,-76.995690,Liceo Naval Almirante Guise,-12.095656,-76.999398,High School
1,"Lima, San Miguel",-12.078656,-77.095283,Colegio Claretiano,-12.073374,-77.087876,School
2,"Lima, San Miguel",-12.078656,-77.095283,Colegio Militar Leoncio Prado,-12.069256,-77.094389,High School
3,"Lima, San Miguel",-12.078656,-77.095283,MAAC,-12.073663,-77.100830,High School
4,"Lima, Rímac",-12.020304,-77.035463,colegio Nuestra Señora De La Consolación,-12.026545,-77.031580,High School
5,"Lima, La Victoria",-12.073358,-77.016417,Colegio Parroquial San Ricardo,-12.072142,-77.022876,High School
6,"Lima, La Victoria",-12.073358,-77.016417,Colegio Isabel La Católica,-12.070306,-77.023952,High School
7,"Lima, La Victoria",-12.073358,-77.016417,Nido Santa Maria,-12.076940,-77.010596,High School
8,"Lima, La Victoria",-12.073358,-77.016417,COLEGIO 1111 - JOSE ANTONIO ENCINAS,-12.066202,-77.011607,High School
9,"Lima, La Victoria",-12.073358,-77.016417,high MR,-12.068783,-77.020297,High School


In [61]:
lima_venues_highschools.shape

(126, 7)

In [62]:
map_lima_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(lima_venues_highschools, 'green', map_lima_highschools)
map_lima_highschools

With this category ID '4bf58dd8d48988d1ae941735' we can visualize the universities per districts

In [63]:
lima_venues_uni = getNearbyVenues(names=lima_df['Distrito'], latitudes=lima_df['Latitude'], longitudes=lima_df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
lima_venues_uni.head(20)

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lima, San Borja",-12.096452,-76.995690,Entrenamiento MikroTik RouterOS by MKE Solutio...,-12.094538,-76.997903,University
1,"Lima, San Borja",-12.096452,-76.995690,UPC - Escuela de Postgrado Sede San Borja,-12.095189,-76.998843,University
2,"Lima, San Borja",-12.096452,-76.995690,EPE - Universidad Peruana de Ciencias Aplicada...,-12.095442,-76.999104,College Academic Building
3,"Lima, San Miguel",-12.078656,-77.095283,Universidad Peruana de Ciencias Aplicadas - UPC,-12.076773,-77.093552,University
4,"Lima, San Miguel",-12.078656,-77.095283,Universidad nacional del callao,-12.073245,-77.088157,University
5,"Lima, San Miguel",-12.078656,-77.095283,"Búnker NG, sede Maranga",-12.072247,-77.097340,University
6,"Lima, San Miguel",-12.078656,-77.095283,Pabellón A - UPC Villa,-12.079104,-77.086201,University
7,"Lima, La Victoria",-12.073358,-77.016417,I.E. Sagrada Familia,-12.066906,-77.021165,University
8,"Lima, La Molina",-12.090177,-76.922338,Clinica Odontologica - USMP,-12.089297,-76.930656,University
9,"Lima, Lima",-12.062107,-77.036526,Universidad Peruana de las Americas,-12.063070,-77.038343,University


In [64]:
map_lima_uni = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(lima_venues_uni, 'gold', map_lima_uni)
map_lima_uni

With this category ID '4d4b7105d754a06375d81259' we can visualize the office markets per districts

In [65]:
lima_venues_office = getNearbyVenues(names=lima_df['Distrito'], latitudes=lima_df['Latitude'], longitudes=lima_df['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
lima_venues_office.head(20)

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lima, San Borja",-12.096452,-76.99569,Centro Naval - Sede San Borja,-12.097401,-76.996091,Event Space
1,"Lima, San Borja",-12.096452,-76.99569,Liceo Naval Almirante Guise,-12.095656,-76.999398,High School
2,"Lima, San Borja",-12.096452,-76.99569,Torre Del Arte,-12.090663,-77.004488,Office
3,"Lima, San Borja",-12.096452,-76.99569,Control Vital,-12.087737,-76.998265,Office
4,"Lima, San Borja",-12.096452,-76.99569,Innova Films,-12.100405,-76.991592,Office
5,"Lima, San Borja",-12.096452,-76.99569,Colegio San Francisco De Borja,-12.090982,-76.997766,School
6,"Lima, San Borja",-12.096452,-76.99569,Anqa - Centro Odontológico,-12.098874,-77.001920,Dentist's Office
7,"Lima, San Borja",-12.096452,-76.99569,Clinica Pulso,-12.087541,-76.997419,Medical Center
8,"Lima, San Borja",-12.096452,-76.99569,Dental Kahn,-12.089430,-76.989741,Dentist's Office
9,"Lima, San Borja",-12.096452,-76.99569,Comisaría San Borja,-12.105704,-76.999127,Police Station


In [74]:
map_lima_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(lima_venues_office, 'fuchsia', map_lima_office)
map_lima_office

Adding into our dataframe

In [69]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0

In [70]:
df_data = lima_df.copy()
df_data.rename(columns={'Distrito':'Localidad'}, inplace=True)
addColumn(df_data, 'Burger', lima_venues_burger)
addColumn(df_data, 'High Schools', lima_venues_highschools)
addColumn(df_data, 'Universities', lima_venues_uni)
addColumn(df_data, 'Offices', lima_venues_office)
df_data.head()

,Ubicación,Localidad,Poblacion,Latitude,Longitude,Burger,High Schools,Universities,Offices
36,150130,"Lima, San Borja",114 688,-12.096452,-76.995690,7.0,1.0,3.0,50.0
28,150122,"Lima, Pachacámac",131 037,-12.250682,-76.906646,2.0,0.0,0.0,8.0
42,150136,"Lima, San Miguel",138 226,-12.078656,-77.095283,20.0,3.0,4.0,50.0
44,150138,"Lima, Santa María del Mar",1638,-12.401403,-76.775465,0.0,0.0,0.0,11.0
34,150128,"Lima, Rímac",167 617,-12.020304,-77.035463,4.0,1.0,0.0,49.0


Finally we will put some weight on each column created

In [75]:
# negative weight, because the person wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_burger = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight, because uni students are good customers
weight_uni = 1.5

# positive weight because employees are even better customers
weight_offices = 2

In [76]:
df_weighted = df_data[['Localidad']].copy()

## 4. Results

In [77]:
df_weighted['Score'] = df_data['Burger'] * weight_burger + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Localidad,Score
19,"Lima, Jesús María",138.0
7,"Lima, Lima",131.0
11,"Lima, Breña",116.0
30,"Lima, Pueblo Libre",113.0
46,"Lima, Santiago de Surco",111.5
27,"Lima, Miraflores",107.0
22,"Lima, Lince",105.0
47,"Lima, Surquillo",100.0
4,"Lima, La Punta",99.5
36,"Lima, San Borja",98.5


In [78]:
map_bog_result = folium.Map(location=[latitude, longitude], zoom_start=15)

bog_win = lima_df[lima_df['Distrito'] == 'Lima, Jesús María']

for lat, lng, local in zip(bog_win['Latitude'], bog_win['Longitude'], bog_win['Distrito']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_bog_result) 

addToMap(lima_venues_burger[lima_venues_burger['Localidad'] == 'Lima, Jesús María'], 'red', map_bog_result)
addToMap(lima_venues_highschools[lima_venues_highschools['Localidad'] == 'Lima, Jesús María'], 'green', map_bog_result)
addToMap(lima_venues_uni[lima_venues_uni['Localidad'] == 'Lima, Jesús María'], 'gold', map_bog_result)
addToMap(lima_venues_office[lima_venues_office['Localidad'] == 'Lima, Jesús María'], 'fuchsia', map_bog_result)

map_bog_result

## 5. Discussion and Conclusion

After analyzing the data we could conclude that Jesus Maria district is the best option to open a new hamburguer restaurant.
# The top 5:
1. Jesus Maria
2. Lima
3. Breña
4. Pueblo Libre
5. Surco